# Prepare SLUrb jobs for model comparison

For SLUrb, jobs with two resolutions are created in order to study grid sensitivity at the same time.: 2 m (fine) and 16 m (coarse).


In [ ]:
%load_ext dotenv
%dotenv
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from src.config import get_config, get_dask_cluster

config = get_config()
cluster, client = get_dask_cluster(config)

In [ ]:
import xarray as xr
import numpy as np

from src.job_generation import (
    Job,
    read_namelist,
    Driver,
    JobNest,
    get_urban_morphology_from_usm_driver,
    set_urban_morphology_to_slurb_driver,
    update_dict_recursive,
    set_radiation_to_dynamic,
    set_initial_soil_conditions_from_precursor,
    set_slurb_deep_soil_temperature_from_dynamic,
    set_spinup_parameters_from_precursor,
    set_surface_pressure_to_dynamic,
    get_nest_mask,
)

## Prepare job objects

The jobs (nests) are constructed by top-down-approach: first a coarse resolution domain is created and the nests are created by upsampling.

### Coarse resolution case


In [ ]:
job_slurb = Job("slurb_c_slurb")

In [ ]:
job_slurb.p3d = read_namelist(
    Path(config.path.experiments.comparison) / "shared_coarse_p3d.yml"
)

Merge case-specific settings


In [ ]:
update_dict_recursive(
    job_slurb.p3d,
    read_namelist(Path(config.path.experiments.comparison) / "slurb_coarse_p3d.yml"),
)

In [ ]:
job_slurb.p3dr = job_slurb.p3d.copy()
job_slurb.p3dr["initialization_parameters"]["initializing_actions"] = (
    "read_restart_data"
)

### Load surface configuration


In [ ]:
lcz_map = xr.open_dataset(Path(config.path.data.raw) / "lcz" / "lcz_map.nc")["lcz"]
usm_driver = xr.open_dataset(Path(config.path.data.raw) / "lcz" / "USM_lcz_fine.nc")

## Setup the coarse domain


### SLUrb driver


In [ ]:
job_slurb.register_driver("slurb", Driver())

In [ ]:
job_slurb.drivers["slurb"].set_grid("s", vertical=False)
job_slurb.drivers["slurb"].set_attrs(
    Path(config.path.experiments.comparison) / "global_attributes.yml"
)
job_slurb.drivers["slurb"].ds = job_slurb.drivers["slurb"].ds.assign_coords(
    nroof_3d=np.arange(1, 5, dtype=np.int8),
    nroad_3d=np.arange(1, 5, dtype=np.int8),
    nwall_3d=np.arange(1, 5, dtype=np.int8),
    nwindow_3d=np.arange(1, 5, dtype=np.int8),
)

Set material parameters from a config file. Note that this file doesn't include the urban morphological parameters, as these are set according to a LCZ.


In [ ]:
job_slurb.driver_from_config(
    Path(config.path.experiments.comparison) / "base_material_params.yml",
    "slurb",
)

Set urban morphology according to a LCZ map.


In [ ]:
urban_params = get_urban_morphology_from_usm_driver(lcz_map, usm_driver)
job_slurb.drivers["slurb"].ds = set_urban_morphology_to_slurb_driver(
    lcz_map.copy(),
    job_slurb.drivers["slurb"].ds,
    urban_params,
    offset_x=2368.0,
    offset_y=1280.0,
    buffer=4,  # four-gridpoint buffer zone
)

In [ ]:
urban_params

### Static driver


For area outside the fine resolution nest, set vegetation as in precursor. All-grass domain would probably be too smooth surface mechanically. First, a child domain mask is needed.


In [ ]:
job_slurb.register_driver("static", Driver())
job_slurb.drivers["static"].ds = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_static"
)
job_slurb.drivers["static"].set_attrs(
    Path(config.path.experiments.comparison) / "global_attributes.yml"
)
job_slurb.drivers["static"].ds["vegetation_pars"][:] = np.nan

Set case-specific modifications to the driver. In SLUrb, the roof height is referenced to the PALM surface, set zt accordingly.


Set terrain height to builiding height or two-thirds of assumed canopy height (20 m) for high vegetation. Furthermore, set vegetation type to grass for the urban area and momentum roughness length to urban value (needed as PALM tiling approach is not yet fully supported).


In [ ]:
job_slurb.drivers["static"].ds["zt"] = (
    ("y", "x"),
    job_slurb.drivers["slurb"].ds["building_height"].data,
)
job_slurb.drivers["static"].ds["zt"] = (
    job_slurb.drivers["static"].ds["zt"].fillna(0.0).astype(np.float32)
)

job_slurb.drivers["static"].ds["vegetation_type"] = (
    job_slurb.drivers["static"]
    .ds["vegetation_type"]
    .where(job_slurb.drivers["slurb"].ds["urban_fraction"] <= 0.0, other=3)
)

job_slurb.drivers["static"].ds["zt"] = (
    job_slurb.drivers["static"]
    .ds["zt"]
    .where(
        ~np.isin(
            job_slurb.drivers["static"].ds["vegetation_type"], [4, 5, 6, 7, 17, 18]
        ),
        other=(2.0 / 3.0) * 20.0,
    )
)
job_slurb.drivers["static"].ds["zt"] = (
    job_slurb.drivers["static"]
    .ds["zt"]
    .where(job_slurb.drivers["static"].ds["x"] <= 3840, other=0.0)
)

Moreover, set _zt_ at the outflow boundary to maximum zt, as the outflow boundary condition (radiation) in combination with the turbulence recycling mass-flux conservation condition is quite sensitive. Do this for the inflow as well to ensure mass flux conservation.


In [ ]:
job_slurb.drivers["static"].ds["zt"][:, -1:] = (
    job_slurb.drivers["slurb"].ds["building_height"].max()
)
job_slurb.drivers["static"].ds["zt"][:, 0] = (
    job_slurb.drivers["slurb"].ds["building_height"].max()
)

In [ ]:
# I don't know why the byte types get casted to into floats when reading the input, but recast them here
int_vars = ("pavement_type", "soil_type", "vegetation_type", "water_type")
job_slurb.drivers["static"].ds["pavement_type"][:] = -127
job_slurb.drivers["static"].ds["water_type"][:] = -127
for var in int_vars:
    job_slurb.drivers["static"].ds[var] = (
        job_slurb.drivers["static"].ds[var].astype(np.int8)
    )

Furthermore, the roughness length is limited to $z_0=0.5*z_{mo}$. This won't affect the coarse resolution runs, but a limitation will be applied in the finer resolution nests. Thus, for consistency, set limited roughness length for all domains. Otherwise, the setups at different resolution levels wouldn't be comparable. The maximum $z_0$ of vegetation lookup tables is $2.0~\mathrm{m}$, so this has to be done only for 4 m and 2 m domain extents.


In [ ]:
medium_fine_mask = (
    (job_slurb.drivers["static"].ds["x"] >= 896.0)
    & (job_slurb.drivers["static"].ds["x"] <= 3968.0)
) & (
    (job_slurb.drivers["static"].ds["y"] >= 512.0)
    & (job_slurb.drivers["static"].ds["y"] <= 3584.0)
)
fine_mask = (
    (job_slurb.drivers["static"].ds["x"] >= 2112.0)
    & (job_slurb.drivers["static"].ds["x"] <= 3904.0)
) & (
    (job_slurb.drivers["static"].ds["y"] >= 1280.0)
    & (job_slurb.drivers["static"].ds["y"] <= 2816.0)
)

job_slurb.drivers["static"].ds["vegetation_pars"][4, :, :] = (
    job_slurb.drivers["static"]
    .ds["vegetation_pars"][4, :, :]
    .where(
        ~medium_fine_mask
        | ~job_slurb.drivers["static"]
        .ds["vegetation_type"]
        .isin([4, 5, 6, 7, 14, 17, 18]),
        other=1.0,
    )
)

job_slurb.drivers["static"].ds["vegetation_pars"][4, :, :] = (
    job_slurb.drivers["slurb"]
    .ds["z0_urb"]
    .where(
        job_slurb.drivers["slurb"].ds["urban_fraction"] > 0.0,
        other=job_slurb.drivers["static"].ds["vegetation_pars"][4, :, :],
    )
)

job_slurb.drivers["static"].ds["vegetation_pars"][5, :, :] = (
    job_slurb.drivers["static"]
    .ds["vegetation_pars"][5, :, :]
    .where(
        ~medium_fine_mask
        | ~job_slurb.drivers["static"]
        .ds["vegetation_type"]
        .isin([4, 5, 6, 7, 14, 17, 18]),
        other=1.0,
    )
)
job_slurb.drivers["static"].ds["vegetation_pars"][4, :, :] = (
    job_slurb.drivers["static"]
    .ds["vegetation_pars"][4, :, :]
    .where(
        ~fine_mask
        | ~job_slurb.drivers["static"]
        .ds["vegetation_type"]
        .isin([4, 5, 6, 7, 14, 17, 18]),
        other=0.5,
    )
)
job_slurb.drivers["static"].ds["vegetation_pars"][5, :, :] = (
    job_slurb.drivers["static"]
    .ds["vegetation_pars"][5, :, :]
    .where(
        ~fine_mask
        | ~job_slurb.drivers["static"]
        .ds["vegetation_type"]
        .isin([4, 5, 6, 7, 14, 17, 18]),
        other=0.5,
    )
)

In [ ]:
job_slurb.drivers["static"].ds["vegetation_type"]

### Set turbulent inflow

For this, we simply reuse the default precursor from the sensitivity tests.


In [ ]:
job_slurb.register_driver("dynamic", Driver())
job_slurb.drivers["dynamic"].set_grid("uvws", vertical=True)
job_slurb.drivers["dynamic"].set_zsoil()
job_slurb.drivers["dynamic"]
job_slurb.set_turbulent_inflow(
    inflow_source_file=Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_yz.001.nc",
    init_source_file=Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_3d.001.nc",
    dynamic_source_file=Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_dynamic",
)

In [ ]:
pre_3d = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_3d.001.nc",
    decode_times=False,
)
pre_static = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "INPUT"
    / "slurb_pre_default_static"
)
job_slurb.drivers["dynamic"] = set_initial_soil_conditions_from_precursor(
    pre_3d,
    pre_static["vegetation_type"],
    job_slurb.drivers["dynamic"],
    job_slurb.drivers["static"].ds["vegetation_type"],
)

job_slurb.drivers["slurb"] = set_slurb_deep_soil_temperature_from_dynamic(
    job_slurb.drivers["slurb"],
    job_slurb.drivers["dynamic"],
)

Modify the initial profiles to ensure that the forcing during the spinup is exactly the same.


In [ ]:
u_init = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_u"]
    .where(job_slurb.drivers["dynamic"].ds["z"] >= 24.0, other=np.nan)
)
v_init = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_v"]
    .where(job_slurb.drivers["dynamic"].ds["z"] >= 24.0, other=np.nan)
)
pt_init = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_pt"]
    .where(job_slurb.drivers["dynamic"].ds["z"] >= 24.0, other=np.nan)
)
qv_init = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_qv"]
    .where(job_slurb.drivers["dynamic"].ds["z"] >= 24.0, other=np.nan)
)
u_init = u_init.bfill(dim="z")
v_init = v_init.bfill(dim="z")
pt_init = pt_init.bfill(dim="z")
qv_init = qv_init.bfill(dim="z")

In [ ]:
pt_init.values

### Spinup parameters


In [ ]:
pre_pr = xr.open_dataset(
    Path(config.path.data.jobs)
    / "slurb_pre_default"
    / "OUTPUT"
    / "slurb_pre_default_pr.001.nc",
    decode_times=False,
)
job_slurb.p3d, job_slurb.p3dr = set_spinup_parameters_from_precursor(
    pre_pr, job_slurb.p3d, job_slurb.p3dr
)

### Radiation forcing


In [ ]:
era5_surf_diurnal = xr.open_dataset(
    config.path.data.raw + "era5/era5_march_2013-2022_surf_diurnal.nc"
)
job_slurb.drivers["dynamic"] = set_radiation_to_dynamic(
    era5_surf_diurnal, job_slurb.drivers["dynamic"], time_offset=3 * 3600
)
job_slurb.drivers["dynamic"].ds["rad_sw_in"].plot()

### Surface pressure


In [ ]:
job_slurb.drivers["dynamic"] = set_surface_pressure_to_dynamic(
    job_slurb.drivers["dynamic"], p0=1e5
)

## Medium-coarse nest


In [ ]:
nest_medium_coarse = JobNest(root=job_slurb, nest_id=2)

### Namelists


In [ ]:
nest_medium_coarse.p3d = read_namelist(
    Path(config.path.experiments.comparison) / "shared_medium_coarse_p3d.yml"
)
update_dict_recursive(
    nest_medium_coarse.p3d,
    read_namelist(
        Path(config.path.experiments.comparison) / "slurb_medium_coarse_p3d.yml"
    ),
)
nest_medium_coarse.p3dr = nest_medium_coarse.p3d.copy()
nest_medium_coarse.p3dr["initialization_parameters"]["initializing_actions"] = (
    "read_restart_data"
)

### Static driver


In [ ]:
# Refined domain for the medium nest by resampling
nest_medium_coarse.register_driver("static", Driver())
nest_medium_coarse.drivers["static"].set_grid("s", vertical=False)
# Offsetting the coordinates is required for reindex_like
nest_medium_coarse.drivers["static"].ds["x"] = (
    nest_medium_coarse.drivers["static"].ds["x"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["static"].ds["y"] = (
    nest_medium_coarse.drivers["static"].ds["y"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][12]
)
nest_medium_coarse.drivers["static"].ds = job_slurb.drivers["static"].ds.reindex_like(
    nest_medium_coarse.drivers["static"].ds, method="nearest"
)
nest_medium_coarse.drivers["static"].ds["x"] = (
    nest_medium_coarse.drivers["static"].ds["x"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["static"].ds["y"] = (
    nest_medium_coarse.drivers["static"].ds["y"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][12]
)

### SLUrb driver


In [ ]:
# Refined domain for the medium-coarse nest by resampling
nest_medium_coarse.register_driver("slurb", Driver())
nest_medium_coarse.drivers["slurb"].set_grid("s", vertical=False)
# Offsetting the coordinates is required for reindex_like
nest_medium_coarse.drivers["slurb"].ds["x"] = (
    nest_medium_coarse.drivers["slurb"].ds["x"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["slurb"].ds["y"] = (
    nest_medium_coarse.drivers["slurb"].ds["y"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][12]
)
nest_medium_coarse.drivers["slurb"].ds = job_slurb.drivers["slurb"].ds.reindex_like(
    nest_medium_coarse.drivers["slurb"].ds, method="nearest"
)
nest_medium_coarse.drivers["slurb"].ds["x"] = (
    nest_medium_coarse.drivers["slurb"].ds["x"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["slurb"].ds["y"] = (
    nest_medium_coarse.drivers["slurb"].ds["y"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][12]
)

### Dynamic driver


For nests, we only need the initial soil profiles.


In [ ]:
nest_medium_coarse.register_driver("dynamic", Driver())
nest_medium_coarse.drivers["dynamic"].set_grid("uvws", vertical=True)
nest_medium_coarse.drivers["dynamic"].set_zsoil()

nest_medium_coarse.drivers["dynamic"].ds["x"] = (
    nest_medium_coarse.drivers["dynamic"].ds["x"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["dynamic"].ds["y"] = (
    nest_medium_coarse.drivers["dynamic"].ds["y"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][12]
)
nest_medium_coarse.drivers["dynamic"].ds["init_soil_t"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_soil_t"]
    .reindex_like(nest_medium_coarse.drivers["dynamic"].ds, method="nearest")
)
nest_medium_coarse.drivers["dynamic"].ds["init_soil_m"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_soil_m"]
    .reindex_like(nest_medium_coarse.drivers["dynamic"].ds, method="nearest")
)
nest_medium_coarse.drivers["dynamic"].ds["x"] = (
    nest_medium_coarse.drivers["dynamic"].ds["x"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][11]
)
nest_medium_coarse.drivers["dynamic"].ds["y"] = (
    nest_medium_coarse.drivers["dynamic"].ds["y"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][12]
)

Interpolate initial state and geostrophic wind speed from the parent.


In [ ]:
nest_medium_coarse.drivers["dynamic"].ds["ls_forcing_ug"] = (
    job_slurb.drivers["dynamic"]
    .ds["ls_forcing_ug"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["ls_forcing_vg"] = (
    job_slurb.drivers["dynamic"]
    .ds["ls_forcing_vg"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_u"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_u"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_v"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_v"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_pt"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_pt"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_qv"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_qv"]
    .interp(
        z=nest_medium_coarse.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"].ds["init_atmosphere_w"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_w"]
    .interp(
        zw=nest_medium_coarse.drivers["dynamic"].ds["zw"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_coarse.drivers["dynamic"] = set_surface_pressure_to_dynamic(
    nest_medium_coarse.drivers["dynamic"], p0=1e5
)

Copy radiation inputs from root.


In [ ]:
nest_medium_coarse.drivers["dynamic"].ds["rad_sw_in"] = job_slurb.drivers["dynamic"].ds[
    "rad_sw_in"
]
nest_medium_coarse.drivers["dynamic"].ds["rad_lw_in"] = job_slurb.drivers["dynamic"].ds[
    "rad_lw_in"
]

### Spinup parameters


In [ ]:
nest_medium_coarse.p3d["initialization_parameters"]["spinup_pt_amplitude"] = (
    job_slurb.p3d["initialization_parameters"]["spinup_pt_amplitude"]
)
nest_medium_coarse.p3d["initialization_parameters"]["spinup_pt_mean"] = job_slurb.p3d[
    "initialization_parameters"
]["spinup_pt_mean"]
nest_medium_coarse.p3dr["initialization_parameters"]["spinup_pt_amplitude"] = (
    job_slurb.p3d["initialization_parameters"]["spinup_pt_amplitude"]
)
nest_medium_coarse.p3dr["initialization_parameters"]["spinup_pt_mean"] = job_slurb.p3d[
    "initialization_parameters"
]["spinup_pt_mean"]

## Medium-fine nest

Repeat the same process as for the medium-coarse nest. I should have wrapped this into an external method.


In [ ]:
nest_medium_fine = JobNest(root=job_slurb, nest_id=3)

Set namelists for the nests.


In [ ]:
nest_medium_fine.p3d = read_namelist(
    Path(config.path.experiments.comparison) / "shared_medium_fine_p3d.yml"
)
update_dict_recursive(
    nest_medium_fine.p3d,
    read_namelist(
        Path(config.path.experiments.comparison) / "slurb_medium_fine_p3d.yml"
    ),
)
nest_medium_fine.p3dr = nest_medium_fine.p3d.copy()
nest_medium_fine.p3dr["initialization_parameters"]["initializing_actions"] = (
    "read_restart_data"
)

### Static driver


In [ ]:
# Refined domain for the medium nest by resampling
nest_medium_fine.register_driver("static", Driver())
nest_medium_fine.drivers["static"].set_grid("s", vertical=False)
# Offsetting the coordinates is required for reindex_like
nest_medium_fine.drivers["static"].ds["x"] = (
    nest_medium_fine.drivers["static"].ds["x"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["static"].ds["y"] = (
    nest_medium_fine.drivers["static"].ds["y"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][19]
)
nest_medium_fine.drivers["static"].ds = job_slurb.drivers["static"].ds.reindex_like(
    nest_medium_fine.drivers["static"].ds, method="nearest"
)
nest_medium_fine.drivers["static"].ds["x"] = (
    nest_medium_fine.drivers["static"].ds["x"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["static"].ds["y"] = (
    nest_medium_fine.drivers["static"].ds["y"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][19]
)

### SLUrb driver


In [ ]:
nest_medium_fine.register_driver("slurb", Driver())
nest_medium_fine.drivers["slurb"].set_grid("s", vertical=False)
nest_medium_fine.drivers["slurb"].ds["x"] = (
    nest_medium_fine.drivers["slurb"].ds["x"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["slurb"].ds["y"] = (
    nest_medium_fine.drivers["slurb"].ds["y"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][19]
)
nest_medium_fine.drivers["slurb"].ds = job_slurb.drivers["slurb"].ds.reindex_like(
    nest_medium_fine.drivers["slurb"].ds, method="nearest"
)
nest_medium_fine.drivers["slurb"].ds["x"] = (
    nest_medium_fine.drivers["slurb"].ds["x"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["slurb"].ds["y"] = (
    nest_medium_fine.drivers["slurb"].ds["y"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][19]
)

### Dynamic driver


In [ ]:
nest_medium_fine.register_driver("dynamic", Driver())
nest_medium_fine.drivers["dynamic"].set_grid("uvws", vertical=True)
nest_medium_fine.drivers["dynamic"].set_zsoil()

nest_medium_fine.drivers["dynamic"].ds["x"] = (
    nest_medium_fine.drivers["dynamic"].ds["x"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["dynamic"].ds["y"] = (
    nest_medium_fine.drivers["dynamic"].ds["y"]
    + job_slurb.p3d["nesting_parameters"]["domain_layouts"][19]
)
nest_medium_fine.drivers["dynamic"].ds["init_soil_t"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_soil_t"]
    .reindex_like(nest_medium_fine.drivers["dynamic"].ds, method="nearest")
)
nest_medium_fine.drivers["dynamic"].ds["init_soil_m"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_soil_m"]
    .reindex_like(nest_medium_fine.drivers["dynamic"].ds, method="nearest")
)
nest_medium_fine.drivers["dynamic"].ds["x"] = (
    nest_medium_fine.drivers["dynamic"].ds["x"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][18]
)
nest_medium_fine.drivers["dynamic"].ds["y"] = (
    nest_medium_fine.drivers["dynamic"].ds["y"]
    - job_slurb.p3d["nesting_parameters"]["domain_layouts"][19]
)

Interpolate initial state and geostrophic wind speed from the parent.


In [ ]:
nest_medium_fine.drivers["dynamic"].ds["ls_forcing_ug"] = (
    job_slurb.drivers["dynamic"]
    .ds["ls_forcing_ug"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["ls_forcing_vg"] = (
    job_slurb.drivers["dynamic"]
    .ds["ls_forcing_vg"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_u"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_u"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_v"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_v"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_pt"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_pt"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_qv"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_qv"]
    .interp(
        z=nest_medium_fine.drivers["dynamic"].ds["z"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"].ds["init_atmosphere_w"] = (
    job_slurb.drivers["dynamic"]
    .ds["init_atmosphere_w"]
    .interp(
        zw=nest_medium_fine.drivers["dynamic"].ds["zw"],
        method="cubic",
        kwargs={"fill_value": "extrapolate"},
    )
)
nest_medium_fine.drivers["dynamic"] = set_surface_pressure_to_dynamic(
    nest_medium_fine.drivers["dynamic"], p0=1e5
)

Copy radiation inputs from root.


In [ ]:
nest_medium_fine.drivers["dynamic"].ds["rad_sw_in"] = job_slurb.drivers["dynamic"].ds[
    "rad_sw_in"
]
nest_medium_fine.drivers["dynamic"].ds["rad_lw_in"] = job_slurb.drivers["dynamic"].ds[
    "rad_lw_in"
]

### Spinup parameters


In [ ]:
nest_medium_fine.p3d["initialization_parameters"]["spinup_pt_amplitude"] = (
    job_slurb.p3d["initialization_parameters"]["spinup_pt_amplitude"]
)
nest_medium_fine.p3d["initialization_parameters"]["spinup_pt_mean"] = job_slurb.p3d[
    "initialization_parameters"
]["spinup_pt_mean"]
nest_medium_fine.p3dr["initialization_parameters"]["spinup_pt_amplitude"] = (
    job_slurb.p3d["initialization_parameters"]["spinup_pt_amplitude"]
)
nest_medium_fine.p3dr["initialization_parameters"]["spinup_pt_mean"] = job_slurb.p3d[
    "initialization_parameters"
]["spinup_pt_mean"]

## Write job files to storage


In [ ]:
job_slurb.write()